# Install, Paths and Parameters

In [1]:
import os
from pathlib import Path
import getpass
import numpy as np
import pandas as pd
import time
import torch
from torch import nn
from torch.utils.data import DataLoader
import torchvision
from tqdm import tqdm
import random
import sys
from collections import defaultdict

from torch.utils.tensorboard import SummaryWriter

# allow imports when running script from within project dir
[sys.path.append(i) for i in ['.', '..']]

# local
# from src.helpers.helpers import get_random_indexes, get_random_classes
from src.model.dino_model import get_dino, ViTWrapper
from src.model.data import *
from src.model.train import *
from src.model.multihead_model import *
from src.helpers.helpers import create_paths

from torchattacks import *
from sklearn import preprocessing

# seed
SEED = 42
random.seed(SEED)
torch.manual_seed(SEED)
np.random.seed(SEED)

DATA_PATH = Path('/','cluster', 'scratch', 'thobauma', 'dl_data')
MAX_PATH = Path('/','cluster', 'scratch', 'mmathys', 'dl_data')

BASE_ADV_PATH = Path(MAX_PATH, 'adversarial_data_tensors')
BASE_POSTHOC_PATH = Path(MAX_PATH, 'posthoc_tensors')
POSTHOC_MODELS_PATH = Path(MAX_PATH, 'posthoc_models')

LINEAR_CLASSIFIER_MODELS_PATH = Path(MAX_PATH, 'linear_classifier_models')

ORI_PATH = Path(DATA_PATH, 'ori')
CLASS_SUBSET_PATH = Path(ORI_PATH, 'class_subset.npy')
CLASS_SUBSET = np.load(CLASS_SUBSET_PATH)

ADV_DATASETS = ['cw', 'fgsm_06', 'pgd_03']

DATASETS = [*ADV_DATASETS, 'ori']

In [2]:
INDEX_SUBSET = None
NUM_WORKERS= 0
PIN_MEMORY=True

BATCH_SIZE = 256
EPOCHS= 3
DEVICE = 'cuda'

In [3]:
DATA_PATHS = create_paths(data_name='ori',
                 datasets_paths=None,  
                 initial_base_path=DATA_PATH, 
                 posthoc_base_path=BASE_POSTHOC_PATH, 
                 train_str='train', 
                 val_str='validation')

for adv_ds in ADV_DATASETS:
    DATA_PATHS = create_paths(data_name=adv_ds,
                 datasets_paths=DATA_PATHS,  
                 initial_base_path=BASE_ADV_PATH, 
                 posthoc_base_path=BASE_POSTHOC_PATH, 
                 train_str='train', 
                 val_str='validation')

In [4]:
DATA_PATHS["cw"]["init"]["train"]["images"]

PosixPath('/cluster/scratch/mmathys/dl_data/adversarial_data_tensors/cw/train/images')

# Import DINO
Official repo: https://github.com/facebookresearch/dino

In [5]:
model, linear_classifier = get_dino()

Please use the `--pretrained_weights` argument to indicate the path of the checkpoint to evaluate.
Since no pretrained weights have been provided, we load the reference pretrained DINO weights.
Model vit_small built.
Embed dim 1536
We load the reference pretrained linear weights.


In [6]:
for k, v in DATA_PATHS.items():
    print(k)
    print(v["init"]["train"]["images"])
    print(v["init"]["train"]["label"])
    

ori
/cluster/scratch/thobauma/dl_data/ori/train/images
/cluster/scratch/thobauma/dl_data/ori/train/labels.csv
cw
/cluster/scratch/mmathys/dl_data/adversarial_data_tensors/cw/train/images
/cluster/scratch/mmathys/dl_data/adversarial_data_tensors/cw/train/labels.csv
fgsm_06
/cluster/scratch/mmathys/dl_data/adversarial_data_tensors/fgsm_06/train/images
/cluster/scratch/mmathys/dl_data/adversarial_data_tensors/fgsm_06/train/labels.csv
pgd_03
/cluster/scratch/mmathys/dl_data/adversarial_data_tensors/pgd_03/train/images
/cluster/scratch/mmathys/dl_data/adversarial_data_tensors/pgd_03/train/labels.csv


# Load data

In [7]:
# Remember to set the correct transformation
# encoder
label_encoder = preprocessing.LabelEncoder()
label_encoder.fit([i for i in CLASS_SUBSET])

loader_dict = defaultdict(dict)

for k, v in DATA_PATHS.items():
    if not k == "ori":
        print(k)
        adv_train_dataset = EnsembleDataset(v["init"]["train"]["images"], 
                                            v["init"]["train"]["label"])
        
        adv_val_dataset = EnsembleDataset(v["init"]["validation"]["images"], 
                                          v["init"]["validation"]["label"])

        loader_dict[k]["train"] = DataLoader(adv_train_dataset, 
                                             batch_size=BATCH_SIZE, 
                                             num_workers=NUM_WORKERS, 
                                             pin_memory=PIN_MEMORY, 
                                             shuffle=True)
        
        loader_dict[k]["validation"] = DataLoader(adv_val_dataset, 
                                             batch_size=BATCH_SIZE, 
                                             num_workers=NUM_WORKERS, 
                                             pin_memory=PIN_MEMORY, 
                                             shuffle=False)
    else:
        clean_train_dataset = ImageDataset(v["init"]["train"]["images"], 
                                           v["init"]["train"]["label"], 
                                           ORIGINAL_TRANSFORM,
                                           CLASS_SUBSET, 
                                           index_subset=None, 
                                           label_encoder=label_encoder)

        clean_val_dataset = ImageDataset(v["init"]["validation"]["images"], 
                                         v["init"]["validation"]["label"],
                                         ORIGINAL_TRANSFORM,
                                         CLASS_SUBSET, 
                                         index_subset=None, 
                                         label_encoder=label_encoder)
        
        loader_dict["ori"]["train"] = DataLoader(clean_train_dataset, 
                              batch_size=BATCH_SIZE, 
                              num_workers=NUM_WORKERS, 
                              pin_memory=PIN_MEMORY, 
                              shuffle=True)
        
        loader_dict["ori"]["validation"] = DataLoader(clean_val_dataset,
                              batch_size=BATCH_SIZE, 
                              num_workers=NUM_WORKERS, 
                              pin_memory=PIN_MEMORY, 
                              shuffle=False)


cw
fgsm_06
pgd_03


In [8]:
loader_dict["cw"]["train"]

In [9]:
version = 'last_hope'

In [10]:
# next(iter(loader_dict["pgd_03"]["train"]))

## Classifier

In [11]:
class LinearClassifier(nn.Module):
    """Linear layer to train on top of frozen features"""
    def __init__(self, dim, num_labels=1000):
        super(LinearClassifier, self).__init__()
        self.num_labels = num_labels
        self.linear = nn.Linear(dim, num_labels)
        self.linear.weight.data.normal_(mean=0.0, std=0.01)
        self.linear.bias.data.zero_()

    def forward(self, x):
        # flatten
        x = x.view(x.size(0), -1)

        # linear layer
        return self.linear(x)


In [12]:
# Linear Binary Classifier
class LinearBC(nn.Module):
    def __init__(self, input_shape):
        super(LinearBC,self).__init__()
        self.fc1 = nn.Linear(input_shape,1)

    def forward(self, x):
        x = torch.sigmoid(self.fc1(x))
        return x

## Train various classifiers on all adversarial datasets

In [ ]:
run_attacks = [
            "ori", 
            "cw",
#             "fgsm_06", 
#             "pgd_03"
]

for attack, loaders in loader_dict.items():
    if attack not in run_attacks:
        print(f'''Skipping for {attack}''')
        continue
        
    # Initialise classifier
    adv_linear_classifier = LinearClassifier(linear_classifier.linear.in_features, 
                                         num_labels=len(CLASS_SUBSET))
    adv_linear_classifier = adv_linear_classifier.cuda()
    
    # Metric logger path
#     LOG_PATH = Path(LOG_BASE_PATH, 'adv_classifier', version, attack)
#     if not os.path.isdir(LOG_PATH):
#         os.makedirs(LOG_PATH)
    
    # train
    pstr = "#"*50 + f''' Training classifier for {attack} ''' + "#"*50
    print(len(pstr)*"#")
    print(pstr)
    print(len(pstr)*"#")
    
    loggers = train(model, 
                    adv_linear_classifier, 
                    loaders["train"], 
                    loaders["validation"], 
                    log_dir=Path(LINEAR_CLASSIFIER_MODELS_PATH, version, attack),
                    tensor_dir=None, 
                    optimizer=None, 
                    adversarial_attack=None,
                    criterion=nn.CrossEntropyLoss(),
                    epochs=1, 
                    val_freq=1, 
                    batch_size=BATCH_SIZE,  
                    lr=0.001, 
                    to_restore = {"epoch": 0, "best_acc": 0.}, 
                    n=4, 
                    avgpool_patchtokens=False, 
                    show_image=False)
    
    # Save adversarial Classifier
    save_path = Path(LINEAR_CLASSIFIER_MODELS_PATH, version, attack)
    
    if not os.path.isdir(save_path):
        os.makedirs(save_path)
        
    save_file_log = f"log_{attack}.pt"
    torch.save(loggers, Path(save_path, save_file_log))
    
    print(f'''Finished Training classifier on {attack}''')

#################################################################################################################################
################################################## Training classifier for ori ##################################################
#################################################################################################################################
Epoch: [0]  [  0/126]  eta: 0:08:12  lr: 0.001000  loss: 4.562117 (4.562117)  time: 3.907374  data: 3.410567  max mem: 1723
Epoch: [0]  [ 20/126]  eta: 0:06:16  lr: 0.001000  loss: 0.287717 (1.009202)  time: 3.537406  data: 3.031131  max mem: 1725
Epoch: [0]  [ 40/126]  eta: 0:05:06  lr: 0.001000  loss: 0.127130 (0.577551)  time: 3.580346  data: 3.073099  max mem: 1725
Epoch: [0]  [ 60/126]  eta: 0:03:55  lr: 0.001000  loss: 0.120224 (0.431370)  time: 3.568691  data: 3.064073  max mem: 1725
Epoch: [0]  [ 80/126]  eta: 0:02:43  lr: 0.001000  loss: 0.089167 (0.348410)  time: 3.550090  data: 3.046097  max 

## Evaluation

### Evaluate on all adversarial datasets

In [ ]:
attacks = [x for x in loader_dict.keys()]

for attack in attacks:
    if attack == "ori":
        pstr = "#"*30 + f''' evaluating adv_classifier trained on {attack} ''' + "#"*30
        print(len(pstr)*"#")
        print(pstr)
        print(len(pstr)*"#")
        adv_classifier = LinearClassifier(linear_classifier.linear.in_features, 
                                 num_labels=len(CLASS_SUBSET))
        adv_classifier.to(DEVICE)

        # load from checkpoint
        log_dir = Path(LINEAR_CLASSIFIER_MODELS_PATH, attack)
        to_restore={'epoch': 1}
        utils.restart_from_checkpoint(
            Path(log_dir, "checkpoint.pth.tar"),
            run_variables=to_restore,
            state_dict=adv_classifier
        )

        for applied_attack in attacks:

            print(">"*5 + f" {applied_attack} dataset ")
            logger_dict, logger = validate_network(model, 
                                                   adv_classifier, 
                                                   loader_dict[applied_attack]["validation"], 
                                                   criterion=nn.CrossEntropyLoss(),
                                                   tensor_dir=None, 
                                                   adversarial_attack=None, 
                                                   n=4, 
                                                   avgpool_patchtokens=False, 
                                                   path_predictions=None)


### Evaluate on newly generated attacks

In [ ]:
attacks = [x for x in loader_dict.keys()]

for attack in attacks:
    
    pstr = "#"*30 + f''' evaluating adv_classifier trained on {attack} ''' + "#"*30
    print(len(pstr)*"#")
    print(pstr)
    print(len(pstr)*"#")
    
    adv_classifier = LinearClassifier(linear_classifier.linear.in_features, 
                             num_labels=len(CLASS_SUBSET))
    adv_classifier.to(DEVICE)
    
    log_dir = Path(LINEAR_CLASSIFIER_MODELS_PATH, attack)
    to_restore={'epoch': 1}
    utils.restart_from_checkpoint(
        Path(log_dir, "checkpoint.pth.tar"),
        run_variables=to_restore,
        state_dict=adv_classifier
    )
    
    vits = ViTWrapper(model, adv_classifier, transform=None)

    for applied_attack in attacks:
        
        if applied_attack == "pgd":
            ev_attack = PGD(vits, eps=0.3, alpha=6/255, steps=15)
        elif applied_attack == "cw":
            ev_attack = CW(vits, c=10, lr=0.003, steps=30)
        elif applied_attack == "fgsm":
            ev_attack = FGSM(vits, eps=0.03)
        else:
            continue
 
        print(">"*5 + f''' applying attack: {ev_attack} ''')
        logger_dict, logger = validate_network(model, 
                                               adv_classifier, 
                                               loader_dict["ori"]["validation"],
                                               criterion=nn.CrossEntropyLoss(),
                                               tensor_dir=None, 
                                               adversarial_attack=ev_attack,
                                               n=4, 
                                               avgpool_patchtokens=False, 
                                               path_predictions=None)

### Evaluate on full pipeline with post-hoc as multiplexer

In [ ]:
# Load clean_classifier
name="clean"
clean_classifier = LinearClassifier(linear_classifier.linear.in_features, 
                                    num_labels=len(CLASS_SUBSET))
clean_classifier.to(DEVICE)

log_dir = Path(LINEAR_CLASSIFIER_MODELS_PATH, "ori")
to_restore={'epoch': 1}
utils.restart_from_checkpoint(
    Path(log_dir, "checkpoint.pth.tar"),
    run_variables=to_restore,
    state_dict=clean_classifier
)

In [ ]:
# Load posthoc
posthocs=["cw", "fgsm_06", "pgd_03"]

adv_models = ["cw", "fgsm_06", "pgd_03"]

# Perform validation on clean dataset
for post_model in posthocs:
    
    log_dir = Path(POSTHOC_MODELS_PATH, post_model)
    
    posthoc = LinearBC(1536)
    posthoc.cuda()
    to_restore={'epoch':3}
    utils.restart_from_checkpoint(
        Path(log_dir, "checkpoint.pth.tar"),
        run_variables=to_restore,
        state_dict=posthoc
    )
    
    for adv_model in adv_models:
        adv_classifier = LinearClassifier(linear_classifier.linear.in_features, 
                                          num_labels=len(CLASS_SUBSET))
        adv_classifier.to(DEVICE)
        
        log_dir = Path(LINEAR_CLASSIFIER_MODELS_PATH, "adv_model")
        to_restore={'epoch': 1}
        
        utils.restart_from_checkpoint(
            Path(log_dir, "checkpoint.pth.tar"),
            run_variables=to_restore,
            state_dict=adv_classifier
        )
        
        for attack, loaders in loader_dict.items():
            pstr = "#"*30 + f''' Validating Posthoc: {post_model} and adv_classifier: {adv_model} on {attack} ''' + "#"*30
            print(len(pstr)*"#")
            print(pstr)
            print(len(pstr)*"#")
            log_dict, logger = validate_multihead_network(model, 
                                                          posthoc,
                                                          adv_classifier,
                                                          clean_classifier,
                                                          loader_dict[attack]["validation"], 
                                                          tensor_dir=None, 
                                                          adversarial_attack=None, 
                                                          n=4, 
                                                          avgpool=False)
            
            # Save adversarial Classifier
            save_path = Path(ADVERSARIAL_CLASSIFIER_MODEL_SAVE_PATH, version, "benchmark")
            if not os.path.isdir(save_path):
                os.makedirs(save_path)
            save_file_log = f"log_post{post_model}_adv{adv_model}_attack{attack}.pt"
            torch.save(logger, str(save_path) + "/" + save_file_log)
            

## Black Box on Multihead model